In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None
import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy

import plotly.express as px

In [3]:
prec_units='mm'
temp_units='C'

scope = cy.Define_Scope()
raw_data = cy.Get_Data_All_Parallel(scope)
raw_data.keys()

dict_keys(['years', 'locations', 'yield', 'weights', 'planting_progress', 'silking_progress', 'w_df_all', 'w_w_df_all'])

In [5]:
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units=prec_units, temp_units=temp_units, ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)

model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [7]:
train_df.columns

Index(['const', 'Trend', 'Yield', 'Planted pct on May 15th', 'Planting Prec',
       'Planting Prec Squared', 'Jul Aug Prec', 'Jul Aug Prec Squared',
       'Prec Interaction', 'Pollination SDD', 'Regular SDD'],
      dtype='object')

In [8]:
model.params

const                     -3720.595956
Trend                         1.850741
Planted pct on May 15th       0.210880
Planting Prec                 0.500343
Planting Prec Squared        -0.000617
Jul Aug Prec                  0.747562
Jul Aug Prec Squared         -0.001017
Prec Interaction             -0.000891
Pollination SDD              -0.195096
Regular SDD                  -0.131014
dtype: float64

In [208]:
step=2.0

col=['Planting Prec']
min_x = train_df[col].min()
max_x = train_df[col].max()

pp_x = np.arange(0,1.0*max_x.values,step)
pp_y = model.params['Planting Prec']*pp_x+ model.params['Planting Prec Squared']*pp_x*pp_x
fig=px.line(x=pp_x, y=pp_y)
# fig.show('browser')
print(pp_x.max())
print(pp_x.shape)


col=['Jul Aug Prec']
min_x = train_df[col].min()
max_x = train_df[col].max()

jp_x = np.arange(0,1.0*max_x.values,step)
jp_y = model.params['Jul Aug Prec']*jp_x+ model.params['Jul Aug Prec Squared']*jp_x*jp_x
fig=px.line(x=jp_x, y=jp_y)
# fig.show('browser')
print(jp_x.max())
print(jp_x.shape)


interaction = np.matrix(pp_x).T * np.matrix(jp_x) * model.params['Prec Interaction']
prec = np.matrix(pp_y).T + np.matrix(jp_y)
all_prec = prec+interaction


fig=px.imshow(all_prec,x=jp_x,y=pp_x)
fig.update_yaxes(autorange=True) 
# fig=px.imshow(all_prec)
fig.show('browser')
print(np.max(all_prec))

322.0
(162,)
394.0
(198,)
155.11337651216758


In [ ]:
date_start=dt(2022,4,1)
date_end=dt(2022,9,30)
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
pred_DF = cy.Build_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
yields=model.predict(pred_DF[model.params.index])
pred_DF['Yield']=yields

In [ ]:
date_start=dt(2022,4,1)
date_end=dt(2022,9,30)
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
int_pred_DF = cy.Build_Intuitive_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end, trend_yield_case=False)
yields=model.predict(int_pred_DF[model.params.index])
int_pred_DF['Yield']=yields

In [ ]:
date_start=dt(2022,4,1)
date_end=dt(2022,9,30)
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
int_base_DF = cy.Build_Intuitive_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end, trend_yield_case=True)
yields=model.predict(int_base_DF[model.params.index])
int_base_DF['Yield']=yields

In [ ]:
date_start=dt(2022,4,1)
date_end=dt(2022,9,30)
ext_dict={'Prec' :'Mean','Sdd30':'Analog_2012'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
int_2012_DF = cy.Build_Intuitive_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end, trend_yield_case=False)
yields=model.predict(int_2012_DF[model.params.index])
int_2012_DF['Yield']=yields

In [ ]:
# Chart 1
df=pd.concat([int_base_DF['Yield'],int_pred_DF['Yield'],int_2012_DF['Yield']],axis=1,keys=['Base','2022','2012'])
fig=px.line(df)
fig.update_layout(hovermode="x unified")
fig.show('browser')

# Chart 2
s_2022=int_base_DF.iloc[-1]['Yield'] +int_pred_DF['Yield']-int_base_DF['Yield']
s_2012=int_base_DF.iloc[-1]['Yield'] +int_2012_DF['Yield']-int_base_DF['Yield']

df=pd.concat([s_2022,s_2012],axis=1,keys=['2022','2012'])
fig=px.line(df)
fig.update_layout(hovermode="x unified")
fig.show('browser')

# Chart 3
s_2022=pred_DF['Yield']
s_2012=pred_DF['Yield'] -(int_pred_DF['Yield']-int_2012_DF['Yield'])

df=pd.concat([s_2022,s_2012],axis=1,keys=['2022','2012'])
fig=px.line(df)
fig.update_layout(hovermode="x unified")
fig.show('browser')

In [ ]:
pred_chart=pd.concat([pred_DF_2012['Yield'],pred_DF['Yield']],axis=1,keys=['2012','Base'])
base_chart=pd.concat([pred_DF_2012['Yield'],pred_DF['Yield']],axis=1,keys=['2012','Base'])
d_pred_chart=pd.concat([d_pred_DF_2012['Yield'],d_pred_DF['Yield']],axis=1,keys=['2012','Base'])


fig=px.line(pred_DF['Yield']-pred_DF_2012['Yield']); fig.show()
fig=px.line(pred_chart); fig.show()
fig=px.line(d_pred_chart); fig.show()
# px.line(df_study)

# px.line(df_study)
